Copyright 2021-2024 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of MindSPONGE:
MindSpore Simulation Package tOwards Next Generation molecular modelling.

MindSPONGE is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

Simulation with trained forcefield.

配置环境, 导入必要的包

In [1]:
import time
import mindspore as ms
import numpy as np
import h5py

import sys
import os
os.environ['MINDSPONGE_HOME']='/home/mindspore/work/summerschool/mindscience/MindSPONGE/src'
os.environ['GLOG_v']=str(4)
path = os.getenv('MINDSPONGE_HOME')
if path:
    sys.path.insert(0, path)
sys.path.append('../..')
data_dir = './data'

from mindspore import context
from mindspore.nn import Adam
from mindspore import Tensor
from mindspore import load_checkpoint

from sponge import Sponge
from sponge import Molecule
from sponge import WithEnergyCell
from sponge import UpdaterMD
from sponge import set_global_units
from sponge.colvar import Distance
from sponge.colvar import ColvarCombine
from sponge.sampling import Metadynamics
from sponge.callback import WriteH5MD, RunInfo
from sponge.function import VelocityGenerator
from sponge.control import VelocityVerlet, Langevin, BerendsenThermostat, LeapFrog
from sponge.potential.bias import UpperWall, LowerWall
from sponge.data import read_yaml

from cybertron.model import MolCT
from cybertron.readout import AtomwiseReadout
from cybertron import CybertronFF

设置单位和模拟的batch数, 步数, 温度, 添加的bias情况, 以及防止文件重复的命名
其中nobias不添加偏置势, metad只添加metadynamics偏置势, bias添加metad和几个键的约束防止体系崩溃

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="Ascend")
set_global_units('nm', 'kj/mol')

simu_batch = 1
simu_step = 1000
simu_temp = 800
simu_bias = 'bias' # nobias, metad, bias
number = '1'

设置模拟体系的原子类型和坐标, 因为坐标从已有的轨迹中直接获得, 这里省略最小化步骤

In [3]:
atom_types = np.array([8,6,6,6,6,6,6,1,1,1,1,1,1,1,1],dtype=np.int32)
atom_types = Tensor(atom_types,dtype=ms.int32)

traj_file = data_dir + '/traj/PES_1-10000-800K-bias-NORMAL0.h5md'
traj = h5py.File(traj_file)
traj_array = np.array(traj['particles/trajectory/position/value'],dtype=float)
rand_idx = np.random.choice(traj_array.shape[0], size=simu_batch, replace=False)
coordinates = traj_array[rand_idx]
rand_coord = Tensor(coordinates, dtype=ms.float32)
system = Molecule(atomic_number=atom_types, coordinate=rand_coord)

加载模型参数

In [4]:
ckpt_file = data_dir + '/ckpt/cybertron-molct-trained-best.ckpt'
config_file = data_dir + '/conf/configure_MolCT_trained.yaml'
config = read_yaml(config_file)
energy = CybertronFF(**config)
_ = load_checkpoint(ckpt_file, energy)

scales = 4.677139 
shifts = -538.89996
_ = energy.set_scaleshift(scale=scales, shift=shifts)

[WARNING] ME(3994237:140702532196160,MainProcess):2024-08-20-14:45:02.121.965 [/home/yuansh/mindsponge/tutorials/summerschool2024/../../../cybertron/cybertron/checkpoint.py:279] For 'load_param_into_net', 3 parameters in the 'net' are not loaded, because they are not in the 'parameter_dict', please check whether the network structure is consistent when training and loading checkpoint.
[WARNING] ME(3994237:140702532196160,MainProcess):2024-08-20-14:45:02.122.574 [/home/yuansh/mindsponge/tutorials/summerschool2024/../../../cybertron/cybertron/checkpoint.py:284] scaleshift._scale is not loaded.
[WARNING] ME(3994237:140702532196160,MainProcess):2024-08-20-14:45:02.122.805 [/home/yuansh/mindsponge/tutorials/summerschool2024/../../../cybertron/cybertron/checkpoint.py:284] scaleshift._shift is not loaded.
[WARNING] ME(3994237:140702532196160,MainProcess):2024-08-20-14:45:02.123.030 [/home/yuansh/mindsponge/tutorials/summerschool2024/../../../cybertron/cybertron/checkpoint.py:284] scaleshift._

设置初始速度

In [5]:
seed = np.random.randint(1, 1000000)
v_gen = VelocityGenerator(simu_temp,seed=seed)
velocity = v_gen(system.coordinate.shape, system.atom_mass)

偏置势设置

In [6]:
d1 = Distance([[5,6]])
d2 = Distance([[0,6]])
d3 = Distance([[0,2]])
d4 = Distance([[2,4]])
d5 = Distance([[3,4]])
d6 = Distance([[3,5]])

combine = ColvarCombine(colvar=[d3,d6],weights=[-1.0, 1.0])
metad = Metadynamics(
    colvar=combine,
    update_pace=20,
    height=4,
    sigma=0.01,
    grid_min=-0.85,
    grid_max=0.45,
    grid_bin=500,
    temperature=300,
    bias_factor=50,
    num_walker=simu_batch,
    share_parameter=False
)
CV_lwall = LowerWall(
    colvar = combine,
    boundary = [-0.86],
    energy_constant = 1000
)
CV_uwall = UpperWall(
    colvar = combine,
    boundary = [0.46],
    energy_constant = 1000
)

u_walls = UpperWall(
    colvar=[d1, d5, d6],
    boundary=[0.18, 0.2, 0.36],
    # depth=[1.0],
    energy_constant=[1000, 1000, 1000]
)

[WARNING] This module will be removed in a future release, please use `CombineCV` instead.


设置控温, 积分器, 模拟步长, 包装模拟用的模型

In [7]:
if simu_bias == 'nobias':
    sim = WithEnergyCell(system, potential=energy)
elif simu_bias == 'metad':
    sim = WithEnergyCell(system, potential=energy, bias=[metad])
elif simu_bias == 'bias':
    sim = WithEnergyCell(system, potential=energy, bias=[metad, u_walls, CV_lwall, CV_uwall])

opt = UpdaterMD(
    system,
    integrator=LeapFrog(system),
    thermostat=Langevin(system, temperature=simu_temp, time_constant=0.5),
    # thermostat=BerendsenThermostat(system, temperature=simu_temp, time_constant=0.5),
    velocity=velocity,
    time_step=1e-3,
)

md = Sponge(sim, optimizer=opt, metrics={'d3': d3, 'd6': d6, 'd2': d2, 'd4': d4})

设置轨迹保存路径, 保存间隔等, 以及输出模拟情况的间隔, 开始模拟

In [8]:
save_path = data_dir + '/traj/'
save_params = str(simu_batch) + '-' + str(simu_step) + '-' + str(simu_temp) + 'K-' + simu_bias + '-' +  number
save_file_name = save_path + save_params + '.h5md'
cb_h5md = WriteH5MD(system, save_file_name, save_freq=10, write_velocity=True, write_force=True)
run_info = RunInfo(200)

beg_time = time.time()
md.run(simu_step, callbacks=[run_info,cb_h5md])

end_time = time.time()

used_time = end_time - beg_time
m, s = divmod(used_time, 60)
h, m = divmod(m, 60)
print ("Run Time: %02d:%02d:%02d" % (h, m, s))

[MindSPONGE] Started simulation at 2024-08-20 14:45:05
[MindSPONGE] Step: 200, E_pot: -7038.9165, E_kin: 169.7032, E_tot: -6869.2134, Temperature: 971.9336, d3: 0.5420631, d6: 0.15189676, d2: 0.114215344, d4: 0.13853967
[MindSPONGE] Step: 400, E_pot: -7073.061, E_kin: 160.00244, E_tot: -6913.0586, Temperature: 916.3748, d3: 0.4429617, d6: 0.1585966, d2: 0.12645057, d4: 0.13184494
[MindSPONGE] Step: 600, E_pot: -7104.879, E_kin: 133.00957, E_tot: -6971.869, Temperature: 761.7797, d3: 0.5770486, d6: 0.14735, d2: 0.12376008, d4: 0.14071643
[MindSPONGE] Step: 800, E_pot: -7116.1777, E_kin: 116.714554, E_tot: -6999.4634, Temperature: 668.45404, d3: 0.45754206, d6: 0.1504721, d2: 0.12224847, d4: 0.13215375
[MindSPONGE] Step: 1000, E_pot: -7114.125, E_kin: 117.18852, E_tot: -6996.9365, Temperature: 671.1686, d3: 0.54846644, d6: 0.14300211, d2: 0.124665536, d4: 0.13829073
[MindSPONGE] Finished simulation at 2024-08-20 14:45:19
[MindSPONGE] Simulation time: 14.44 seconds.
----------------------